
list wikimedia articles?

ROUTE = /wikimedia-search.json


In [ ]:
const readIndex = importer.import('load wikimedia index')

let allTitles = []


async function readWikimedia(search) {

  let matches = []
  if(search) {
    search = search.toLocaleLowerCase().split(/,|\s+/)
  }

  // use memory cache
  if (allTitles.length) {
    for (let i = 0; i < allTitles.length; i++) {
      let match = search.filter(s => allTitles[i].title.includes(s)).length
      if (search && match >= search.length - 1 && match > 0) {
        console.log('match found in set:', search, allTitles[i].title)
        matches.push(allTitles[i])
      }
    }
    return matches
  }

  // === Step 3: Tie it all together ===
  await new Promise(resolve => readIndex(search, ({ offset, length }, titles, finished) => {
    console.log('reading index:', finished, titles[0])
    for (let i = 0; i < titles.length; i++) {
      allTitles.push({ offset, length, title: titles[i] })
      let match = search.filter(s => titles[i].includes(s)).length 
      if (search && match >= search.length - 1 && match > 0) {
        console.log('match found in set:', search, titles[i])
        matches.push({ offset, length, title: titles[i] })
      }
    }

    if (finished === true) {
      resolve()
    }
  }));

  return matches

}

module.exports = readWikimedia



load wikimedia index?


In [ ]:
const fs = require('fs');
const path = require('path')
//const zlib = require('zlib');

const INDEX_FILE = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads', 'enwiki-20250420-pages-articles-multistream-index.txt')
const XML_FILE = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads', 'enwiki-20250420-pages-articles-multistream.xml.bz2')

function readIndex(search, callback) {
  const length = fs.statSync(INDEX_FILE).size
  const stream = fs.createReadStream(INDEX_FILE, {
    highWaterMark: 64 * 1024,
  });
  let lastOffset
  let leftover = ''
  let lastTitles = []
  let offsetCount = 0
  stream.on('data', chunk => {
    offsetCount += chunk.length
    const lines = (leftover + chunk.toString()).split('\n');
    leftover = lines.pop();
    for (const line of lines) {
      const [offsetStr, pageId, title] = line.split(':');
      const currentOffset = parseInt(offsetStr)
      if (currentOffset && currentOffset !== lastOffset) {
        if (lastOffset) {
          callback({ length: currentOffset - lastOffset, offset: lastOffset }, lastTitles, offsetCount / length * 100.0);
        }
        lastOffset = currentOffset
        lastTitles = []
      }
      lastTitles.push(title.toLocaleLowerCase())
    }
  });

  stream.on('end', () => {
    callback({ length: fs.statSync(XML_FILE).size - lastOffset, offset: lastOffset }, lastTitles, true);
  });
}

module.exports = readIndex



extract wikimedia chunk?


In [ ]:
const fs = require('fs');
const path = require('path')
//const zlib = require('zlib');
const { PassThrough, Transform } = require('stream');
const XmlStream = require('xml-stream');
const bz2 = require('unbzip2-stream');

const XML_FILE = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads', 'enwiki-20250420-pages-articles-multistream.xml.bz2')

// Extract and parse one chunk
function extractChunk(startOffset, endOffset, callback) {
  console.log('reading:', startOffset, endOffset)
  const fileStream = fs.createReadStream(XML_FILE, {
    start: startOffset,
    end: endOffset - 1 // end is inclusive
  });

  const decompress = bz2();

  // 2. Create a Transform stream to replace `\n`
  const normalizeNewlines = new Transform({
    decodeStrings: false,
    transform(chunk, encoding, callback) {
      // Replace or remove newlines here:
      const text = chunk.toString().replace(/\n/g, '&lt;br /&gt;'); // or replace(/\n/g, ' ')
      this.push(text);
      callback();
    }
  });
  
  // 3. Compose stream: wrapperStart + transformed decompressed + wrapperEnd
  const wrapperStart = new PassThrough();
  wrapperStart.end('<mediawiki>');
  
  const wrapperEnd = new PassThrough();
  wrapperEnd.end('</mediawiki>');
  
  const decompressedStream = fileStream.pipe(decompress).pipe(normalizeNewlines);
  
  const fullStream = new PassThrough();
  wrapperStart.pipe(fullStream, { end: false });
  decompressedStream.pipe(fullStream, { end: false });
  decompressedStream.on('end', () => {
    wrapperEnd.pipe(fullStream);
  });
  
  // 4. Pipe into XmlStream
  const xml = new XmlStream(fullStream);

  xml.preserve('text'); // This is a key part
  xml.collect('text');
  
  xml.on('endElement: page', page => {
    //console.log(`Title: ${page.title}`);
    callback(page)
  });

  xml.on('end', () => {
    console.log(`Chunk finished\n`);
    callback(false);
  });

  xml.on('error', err => {
    console.error('XML error:', err);
  });

  decompress.on('error', err => {
    console.error('BZ2 decompression error:', err);
  });
}

module.exports = extractChunk



wikimedia-page.html?

ROUTE = /wikimedia-page.html


In [ ]:
const fs = require('fs')
const extractChunk = importer.import('extract wikimedia chunk')
const preprocessDoc = importer.import('mediawiki text preprocessor')
//const wtf = require('wtf_wikipedia')
const Mustache = require('mustache');
//const parsoid = require('parsoid');

async function extractWikimedia(offset, length, search) {
  if (!search) {
    throw new Error('Page not found.')
  }
  let markdown = ''
  let xmlPage
  offset = parseInt(offset)
  length = parseInt(length)
  try {
    await new Promise(resolve => extractChunk(offset, offset + length, page => {
      if (page === false) {
        return resolve()
      }

      if (page.title.toLocaleLowerCase().includes(search.toLocaleLowerCase())) {
        //console.log(page.revision.text[0]['$children'].slice(100))
        xmlPage = page
        markdown = page.revision.text[0]['$children'].join('').replaceAll('<br />', '\n') // for parsing
      }
    }))

    //let doc = wtf()
    let template = importer.interpret('wikimedia mustache template').code
    //console.log(markdown)
    fs.writeFileSync(__dirname + '/../test.md', markdown)
    const preprocessed = preprocessDoc(markdown);
    const content = Mustache.render(template, Object.assign({
      title: xmlPage.title
    }, preprocessed)); //await wikitextToHTML(markdown)
    const html = Mustache.render(importer.interpret('wikiemedia clone index').code, {
      content: content,
      TIMESTAMP: Date.now()
    });
    return html
  } catch (e) {
    throw e
  }
}



module.exports = extractWikimedia



mediawiki text preprocessor?



In [ ]:
const balanced = importer.import('balanced')
const Mustache = require('mustache');
const { Remarkable } = require('remarkable');
const md = new Remarkable({ html: true, xhtmlOut: true, breaks: true });

function preprocessDoc(wikiText) {
  //const templateMatcher = /\{\{\s*(#?[a-zA-Z0-9_]+)(.*?)\}\}/gs;
  let remaining = wikiText
  let count = 0
  const templates = []
  let remainingStr = ''
  while (true) {
    const match = balanced('{{', '}}', remaining);
    if (!match) {
      remainingStr += remaining
      break;
    }
    templates.push(match)
    remainingStr += remaining.substring(0, match.start) + '{{template' + count + '}}'
    remaining = remaining.substring(match.end + 2, remaining.length)
    count++
  }
  //const templates = coalesceByPattern(wikiText, templateMatcher, matchTemplate);

  //const remaining = templates.map((t, i) => '&template' + i + ';' + t.content.substr(t.raw.length)).join('')

  const tags = ["ref", "references", "syntaxhighlight", "code", "pre", "math", "gallery", "poem"];
  const tagRegex = new RegExp(
    `<(${tags.join("|")})\\b([^>]*)\\s*(\\/>)|<(${tags.join("|")})\\b([^>]*)>([\\s\\S]*?)<\\/\\4>`,
    "gi"
  );
  const pageTags = coalesceByPattern(remainingStr, tagRegex, matchTag)

  //console.log(pageTags[0])
  const linksFuncs = {}

  const pageHtmlTags = pageTags.map((t, i) => {
    if (t.raw.length === 0) {
      return t.content
    }
    linksFuncs['tag' + i] = t.raw.replaceAll('<', '&lt;').replaceAll('>', '&gt;')
    return '{{tag' + i + '}}' + t.content.substr(t.raw.length)
  }).join('')

  const links = []
  remaining = pageHtmlTags
  count = 0
  while (true) {
    const match = balanced('[[', ']]', remaining);
    if (!match) break;
    let newLink = matchWikiLinks((/(?:(\w{2,12}):)?([^\|\]]+)(?:\|([^\]]+))?/gi).exec(match.body))
    links.push(newLink)
    remaining = remaining.substring(0, match.start) + '{{{link' + count + '}}}' + remaining.substring(match.end + 2, remaining.length)
    if (newLink.type === 'file') {

    } else {
      linksFuncs['link' + count] = `<a href="/wiki/${newLink.target}">${newLink.label}</a>`
    }
    count++
  }

  const linkRegex = /\[(https?):\/\/(?:\s([^\]]+))?(?:[\|\s]([^\]]+))?\]/g;
  const pageLinks = coalesceByPattern(remaining, linkRegex, matchWikiLinks)

  const pageHtmlLinks = pageLinks.map((t, i) => {
    if (t.raw.length === 0) {
      return t.content
    }
    linksFuncs['link' + (count + i)] = `<a href="${t.target}">${t.label}</a>`
    return '{{{link' + (count + i) + '}}}' + t.content.substr(t.raw.length)
  }).join('')

  const tablePattern = /\{\|[\s\S]*?\|\}/g;

  const tables = coalesceByPattern(pageHtmlLinks, tablePattern, matchTable)


  const pageHtmlTables = tables.map((t, i) => {
    return '{{table|' + i + '}}' + t.content.substr(t.raw.length)
  }).join('')


  const headingRegex = /(\={1,6})\s*(.*?)\s*\1\s*/gm;

  // finally removed everything that can interfere with sections headings equal signs
  const sections = coalesceByPattern(pageHtmlTables, headingRegex, matchSection);

  //console.log(sections)

  return {
    sections: sections.map(s => ({
      depth: s.depth,
      title: s.title,
      content: md.render(formatWikitextSimple(Mustache.render(
        s.content.substr(s.raw.length)

          // Definition list ;term \n :definition (ignore newlines, flatten first)
          .replaceAll(/;[ \t]*(.+?)[ \t]*\n?:[ \t]*(.+?)(?=\n|$)/g, (_, term, def) => {
            return `<dl><dt>${term}</dt><dd>${def}</dd></dl>`;
          })

          // Blockquote: > lines (even in the middle of paragraphs)
          .replaceAll(/(^|\n)[ \t]*>[ \t]*(.*)/g, '$1<blockquote>$2</blockquote>')
        , Object.assign({}, linksFuncs))))
      //content: JSON.stringify()
    })),
  };
}

function formatWikitextSimple(wikiText) {
  console.log(wikiText)
  return wikiText
    // Horizontal rule (----)
    .replaceAll(/-{4,}/g, '<hr />')

    // Bold + Italic: '''''text'''''
    .replaceAll(/'''''(.*?)'''''/g, '<b><i>$1</i></b>')

    // Bold: '''text'''
    .replaceAll(/'''(.*?)'''/g, '<b>$1</b>')

    // Italic: ''text''
    .replaceAll(/''(.*?)''/g, '<i>$1</i>')

    // Unordered list: one or more * with optional spacing
    .replaceAll(/(^|\n|[ \t])[ \t]*\*+[ \t]*([^\*\n]+)/g, (_, _2, item) => `<ul><li>${item}</li></ul>`)

    // Ordered list: one or more # with optional spacing
    .replaceAll(/(^|\n|[ \t])[ \t]*\#+[ \t]*([^\#\n]+)/g, (_, _2, item) => `<ol><li>${item}</li></ol>`)
    .replaceAll(/<\/ul>\s*<ul>/g, '')
    .replaceAll(/<\/ol>\s*<ol>/g, '')

  //.replaceAll('\n\n', '\n<br />\n')
}

function convertToParagraphs(wikitext) {
  return wikitext
    // Normalize line endings and trim
    .replace(/\r\n?/g, '\n')
    .trim()

    // Collapse multiple blank lines
    .replace(/\n{3,}/g, '\n\n')

    // Convert double line breaks into paragraph breaks
    .split(/\n\s*\n/)
    .map(block => `<p>${block.trim()}</p>`)
    .join('\n\n');
}
function matchTable(match) {
  const tableText = match[0];
  const lines = tableText.split(/\r?\n/).map(line => line.trim());
  const table = {
    caption: null,
    rows: [],
  };

  let currentRow = [];

  for (let line of lines) {
    if (line.startsWith('{|')) continue;
    if (line.startsWith('|}')) {
      if (currentRow.length) table.rows.push(currentRow);
      break;
    }

    if (line.startsWith('|+')) {
      table.caption = line.substring(2).trim();
    } else if (line.startsWith('|-')) {
      if (currentRow.length) table.rows.push(currentRow);
      currentRow = [];
    } else if (line.startsWith('!')) {
      // Header cells
      const headers = line.substring(1).split(/!!/g).map(h => h.trim());
      currentRow.push(...headers.map(text => ({ type: 'th', text })));
    } else if (line.startsWith('|')) {
      // Regular cells
      const cells = line.substring(1).split(/\|\|/g).map(c => c.trim());
      currentRow.push(...cells.map(text => ({ type: 'td', text })));
    }
  }

  return table
}

function matchWikiLinks(match) {
  const langOrNs = match[1];
  const target = match[2];
  const label = match[3] || match[2];

  let type = "internal";
  if (langOrNs === "Category") type = "category";
  if (langOrNs === "File") type = "file";
  if (langOrNs === 'http') type = "url";
  else if (langOrNs && langOrNs.length === 2) type = "interwiki";

  return {
    type,
    lang: type === "interwiki" ? langOrNs : null,
    target,
    label,
  };
}

function matchTag(match) {
  const isSelfClosing = !!match[3];
  const tagName = match[1] || match[4];
  const attrs = match[2] || match[5] || "";
  const content = isSelfClosing ? null : match[6];

  return {
    tag: tagName.toLowerCase(),
    attributes: attrs.trim(),
    inner: content,
  };
}

function matchSection(match) {
  const depth = match[1].length;
  const title = match[2].trim();
  return { depth, title };
}

function coalesceByPattern(text, regex, extractLabel = (match) => match[0]) {
  const matches = [...text.matchAll(regex)];
  const sections = [];

  if (matches.length === 0) {
    return [{ offset: 0, match: null, content: text, raw: '' }];
  } else {
    sections.push({
      offset: 0,
      match: null,
      content: text.slice(0, matches[0].index),
      raw: ''
    })
  }

  for (let i = 0; i < matches.length; i++) {
    const current = matches[i];
    const next = matches[i + 1];

    const startOffset = current.index;
    const endOffset = next ? next.index : text.length;

    sections.push(Object.assign({}, extractLabel(current), {
      offset: startOffset,
      match: current,
      content: text.slice(startOffset, endOffset),
      raw: current[0]
    }));
  }

  return sections;
}

function matchTemplate([full, name, args]) {
  const type = getTemplateType(name);
  return {
    type,
    name,
    args: args.trim(),
  };
}

function getTemplateType(name) {
  if (name.startsWith('#')) return 'parser_function';
  if (name.toUpperCase() === name && name.length > 3) return 'magic_word';
  if (/^infobox/i.test(name)) return 'infobox';
  return 'template';
}


module.exports = preprocessDoc



wikimedia mustache template?


In [ ]:
<h1>{{title}}</h1>
<p><strong>Page ID:</strong> {{pageID}}</p>

{{#categories.length}}
<p><strong>Categories:</strong> {{#categories}}<span>{{.}}</span> {{/categories}}</p>
{{/categories.length}}

{{#sections}}
<h{{depth}}>{{title}}</h{{depth}}>

{{{content}}}

{{#paragraphs}}
<p>
  {{#sentences}}
  {{#text}}
  {{{.}}}
  {{/text}}
  {{/sentences}}
</p>
{{/paragraphs}}

{{#templates.length}}
<h4>Templates</h4>
<ul>
  {{#templates}}
  <li>{{{template}}}: {{{prop}}}</li>
  {{/templates}}
</ul>
{{/templates.length}}

{{#infoboxes}}
<h4>Infoboxes</h4>
{{{infoboxes}}}
{{/infoboxes}}

{{#references.length}}
<h4>References</h4>
<ul>
  {{#references}}
  <li>{{type}}: {{title}}</li>
  {{/references}}
</ul>
{{/references.length}}

{{/sections}}


wikimedia service?

ROUTE[] = /wiki/:article

ROUTE[] = /wiki

ROOT = true

DEFAULT = true


In [ ]:
const readWikimedia = importer.import('list wikimedia articles')
const levSearch = importer.import('search levenshtein distance')
const levDist = importer.import('find the levenshtein distance')
//const extractWikimedia = importer.import('wikimedia-page.html')
const Mustache = require('mustache');

async function searchWikimedia(req, res, next) {
  let article = req.body['article'] || req.params['article'] || req.query['article'] || req.cookies['article']
  let searchResults = await readWikimedia(article)

  if (searchResults.length === 0) {
    throw new Error('Page not found: ' + article)
  }

  console.log('sorting:', searchResults.length)
  let sorted = levSearch(searchResults, { keys: ['title'] }, article)

  console.log(sorted[0].title, levDist(sorted[0].title, article), sorted[1].title, levDist(sorted[1].title, article))
  // check for multiple good matches and display a list
  if ((sorted[0] && levDist(sorted[0].title, article) / article.length < 0.2
    && sorted[1] && levDist(sorted[1].title, article) / article.length < 0.2)
    // no great matches
    || (levDist(sorted[0].title, article) / article.length > 0.5)
  ) {
    let template = importer.interpret('wikimedia mustache template').code
    const content = Mustache.render(`
<h1>Search results</h1>

{{#sections}}
<h2><a href="/wikimedia-page.html?offset={{offset}}&length={{length}}&search={{title}}">{{title}}</a></h2>
{{/sections}}
`, {
      sections: sorted
    });
    const html = Mustache.render(importer.interpret('wikiemedia clone index').code, {
      content: content,
      TIMESTAMP: Date.now()
    });
    return res.send(html)
  }

  //let content = extractWikimedia(sorted[0].offset, sorted[0].length, sorted[0].title)
  return res.redirect('/wikimedia-page.html?offset=' + sorted[0].offset + '&length=' + sorted[0].length + '&search=' + sorted[0].title)
}

module.exports = searchWikimedia



wikiemedia clone index?


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>Shleppedia</title>
  <link rel="stylesheet" href="/wikimedia-style.css?t={{TIMESTAMP}}">
</head>

<body>

  <header>
    <h1>Shleppedia</h1>
    <form id="search-box" method="get" action="/wiki">
      <input name="article" type="text" placeholder="Search articles...">
    </form>
    <p>The slightly off-brand, slightly cooler encyclopedia clone</p>
  </header>


  <div id="main-content">
    <!-- Mustache-rendered content goes here -->
    {{{ content }}}

    <div class="infobox">
      <details open>
        <summary>🧠 Infobox Decoder</summary>
        <div id="infobox-area">
          <!-- Dynamically rendered infobox properties here -->
          <p>No infoboxes loaded yet.</p>
        </div>
      </details>
    </div>
  </div>

</body>

</html>


wikimedia-style?

ROUTE = /wikimedia-style.css


In [ ]:
body {
  margin: 0;
  padding: 0;
  font-family: system-ui, sans-serif;
  background: #f9f9f9;
  color: #333;
}

header {
  background: #4b8bbe;
  color: white;
  padding: 1rem 2rem;
  display: flex;
  align-items: center;
  justify-content: space-between;
  flex-wrap: wrap;
  box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

header h1 {
  font-size: 1.8rem;
  margin: 0;
  flex: 0;
}

#search-box {
  flex: 2;
  max-width: 400px;
  display: flex;
  justify-content: flex-end;
}

#search-box input {
  width: 100%;
  padding: 0.5rem 0.75rem;
  font-size: 1rem;
  border: 1px solid #ccc;
  border-radius: 4px;
  outline: none;
  transition: border-color 0.3s;
  background: white;
}

#search-box input:focus {
  border-color: #2e6fa3;
}

#main-content {
  width: 100%;
  max-width: 1200px;
  margin: 2rem auto;
  padding: 2rem;
  background: white;
  border-radius: 8px;
  box-shadow: 0 0 8px rgba(0, 0, 0, 0.05);
}

.infobox {
  margin-top: 2rem;
  padding: 1rem;
  border-left: 4px solid #4b8bbe;
  background: #eef6ff;
}

details summary {
  cursor: pointer;
  font-weight: bold;
}

a {
  color: #2970b8;
  text-decoration: none;
}

a:hover {
  text-decoration: underline;
}
